In [ ]:
!git clone https://github.com/LeGiangK62/Het-Net-GNN.git
%cd Het-Net-GNN/Main

Cloning into 'Het-Net-GNN'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 64 (delta 22), reused 52 (delta 13), pack-reused 0
Receiving objects: 100% (64/64), 140.11 KiB | 2.46 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/Het-Net-GNN/Main


In [ ]:
%%capture
import torch
!pip install torch_geometric

# Optional dependencies:
if torch.cuda.is_available():
  !
else:
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
#

In [ ]:
import torch
import numpy as np

from torch_geometric.data import HeteroData
from torch_geometric.loader import DataLoader


from WSN_GNN import generate_channels_wsn
from hgt_conv import HGTGNN
from het_net_gnn import RGCN

# Create HeteroData from the wireless system

In [29]:
import numpy as np

# Suppose p_max is a variable in your code
p_max = 5.0  # Example value, replace this with your variable

# Convert p_max to a NumPy array
p_max_array = np.array(1)
print(p_max_array)

# Check if p_max_array is a scalar or an array
if np.isscalar(p_max):
    print("p_max is a scalar value.")
else:
    print("p_max is a NumPy array.")


1
p_max is a scalar value.


In [30]:
#region Create HeteroData from the wireless system
def convert_to_hetero_data(channel_matrices, p_max):
    graph_list = []
    num_sam, num_aps, num_users = channel_matrices.shape
    for i in range(num_sam):
        x1 = torch.ones(num_users, 1) * p_max
        x2 = torch.zeros(num_users, 1)  # power allocation
        x3 = torch.ones((num_users, 1))  # ap selection?
        user_feat = torch.cat((x1,x2,x3),1)  # features of user_node
        ap_feat = torch.ones(num_aps, num_aps_features)  # features of user_node
        edge_feat_uplink = channel_matrices[i, :, :].reshape(-1, 1)
        edge_feat_downlink = channel_matrices[i, :, :].reshape(-1, 1)
        graph = HeteroData({
            'ue': {'x': user_feat},
            'ap': {'x': ap_feat}
        })
        # Create edge types and building the graph connectivity:
        graph['ue', 'uplink', 'ap'].edge_attr = torch.tensor(edge_feat_uplink, dtype=torch.float)
        graph['ap', 'downlink', 'ue'].edge_attr = torch.tensor(edge_feat_downlink, dtype=torch.float)

        graph['ue', 'uplink', 'ap'].edge_index = torch.tensor(adj_matrix(num_users, num_aps).transpose(),
                                                              dtype=torch.int64).contiguous()
        graph['ap', 'downlink', 'ue'].edge_index = torch.tensor(adj_matrix(num_aps, num_users).transpose(),
                                                                dtype=torch.int64).contiguous()

        # Swap
        # graph['ue', 'uplink', 'ap'].edge_index = torch.tensor(adj_matrix(num_aps, num_users).transpose(),
        #                                                       dtype=torch.int64)
        # graph['ap', 'downlink', 'ue'].edge_index = torch.tensor(adj_matrix(num_users, num_aps).transpose(),
        #                                                         dtype=torch.int64)

        # graph['ap', 'downlink', 'ue'].edge_attr  = torch.tensor(edge_feat_downlink, dtype=torch.float)
        graph_list.append(graph)
    return graph_list


def adj_matrix(num_from, num_dest):
    adj = []
    for i in range(num_from):
        for j in range(num_dest):
            adj.append([i, j])
    return np.array(adj)

# Build Heterogeneous GNN

In [ ]:
from typing import Dict, List, Optional, Union

import torch
import torch.nn as nn
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, Sigmoid, Softmax, BatchNorm1d as BN
from torch import Tensor
import torch.nn.functional as F

from torch_geometric.nn.conv import MessagePassing, HeteroConv
from torch_geometric.nn.module_dict import ModuleDict
from torch_geometric.typing import EdgeType, Metadata, NodeType, SparseTensor
from torch_geometric.nn.inits import reset

In [ ]:
def mlp(channels, batch_norm=True):
    return Seq(*[
        Seq(Lin(channels[i - 1], channels[i], bias=True), ReLU())  # , BN(channels[i]))
        for i in range(1, len(channels))
    ])


class EdgeConv(MessagePassing):
    def __init__(self, node_dim, edge_dim, metadata: Metadata, aggr='mean', **kwargs):
        super(EdgeConv, self).__init__(aggr=aggr)

        self.lin_node = ModuleDict()
        self.lin_edge = ModuleDict()
        self.res_lin = ModuleDict()

        for node_type in metadata[0]:
            self.lin_node[node_type] = mlp([node_dim, 32])
            self.res_lin[node_type] = nn.Linear(node_dim, 32)

        for edge_type in metadata[1]:
            self.lin_edge['__'.join(edge_type)] = mlp([edge_dim, 32])

        self.power_mlp = mlp([32 + 3, 16])
        self.power_mlp = Seq(*[self.power_mlp, Seq(Lin(16, 1, bias=True), Sigmoid())])

        self.ap_mlp = mlp([32 + 3, 16])
        self.ap_mlp = Seq(*[self.ap_mlp, Seq(Lin(16, 1, bias=True), Sigmoid())])

        # self.power_mlp = mlp([32 + 3, 1])
        # self.ap_mlp = mlp([32 + 3, 1])



        self.reset_parameters()

    def reset_parameters(self):
        super().reset_parameters()
        reset(self.lin_node)
        reset(self.lin_edge)
        reset(self.res_lin)
        reset(self.power_mlp)
        reset(self.ap_mlp)

    def forward(
        self,
        x_dict: Dict[NodeType, Tensor],
        edge_index_dict: Union[Dict[EdgeType, Tensor],
                               Dict[EdgeType, SparseTensor]],  # Support both.
        edge_attr_dict: Union[Dict[EdgeType, Tensor],
                              Dict[EdgeType, SparseTensor]]
    ) -> Dict[NodeType, Optional[Tensor]]:
        # How to get the edge attributes from only the index?
        lin_node_dict, lin_edge_dict, out_dict = {}, {}, {}
        # Iterate over node-types to initialize the output dictionary
        for node_type, node_ft in x_dict.items():
            lin_node_dict[node_type] = self.lin_node[node_type](node_ft)
            out_dict[node_type] = []

        # Iterate over edge-types:
        for edge_type, edge_index in edge_index_dict.items():
            # aggregate information to the destination node
            # for each type of edge
            edge_attr = edge_attr_dict[edge_type]
            src_type, _, dst_type = edge_type
            edge_type = '__'.join(edge_type)
            x_j = x_dict[src_type]
            x_i = x_dict[dst_type]

            out = self.propagate(edge_index, x=x_j, node_feat=x_i, src_type=src_type, edge_type=edge_type,
                                 dst_type=dst_type, edge_attr=edge_attr, size=(x_j.shape[0], x_i.shape[0]))
            out_dict[dst_type] = out

        # Iterate over node-types:
        # for node_type, outs in out_dict.items():
            # out = group(outs, self.group)
            #
            # if out is None:
            #     out_dict[node_type] = None
            #     continue
            #
            # out = self.a_lin[node_type](F.gelu(out))
            # if out.size(-1) == x_dict[node_type].size(-1):
            #     alpha = self.skip[node_type].sigmoid()
            #     out = alpha * out + (1 - alpha) * x_dict[node_type]
            # out_dict[node_type] = out

        return out_dict

    # def message(self, x_j: Tensor, src_type, edge_type, node_mlp, edge_rel) -> Tensor:
    def message(self, x_j: Tensor, src_type, edge_type, edge_attr) -> Tensor:
        # This function is called when we use self.propagate - Used the given parameters too.
        # What each neighbor node send to target along the edges
        # Adding the edge relation here
        node_mlp = self.lin_node[src_type]
        edge_mlp = self.lin_edge[edge_type]
        return node_mlp(x_j) + edge_mlp(edge_attr)

    def update(self, aggr_out, node_feat, dst_type, edge_index):
        # Update node representations with the aggregated messages
        # aggr_out  = output of aggregation function, the following is the input of the propagation function
        power_max = node_feat[:, 0]
        node_mlp = self.lin_node[dst_type]
        res = node_mlp(node_feat)
        tmp = torch.cat([node_feat, aggr_out + res], dim=1)
        power = self.power_mlp(tmp)
        ap_selection = self.ap_mlp(tmp)
        return torch.cat([power_max.unsqueeze(-1), power, ap_selection], dim=1)


class RGCN(nn.Module):
    def __init__(self, dataset, num_layers):
        # The only things need to fix here are the dimensions
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.mlp = mlp([32, 16])
        for _ in range(num_layers):
            conv = EdgeConv(node_dim=3, edge_dim=1,
                            metadata=dataset.metadata())
            self.convs.append(conv)


    def forward(self, x_dict, edge_index_dict, edge_attr_dict):
        for conv in self.convs:
            x_dict = conv(x_dict=x_dict, edge_index_dict=edge_index_dict, edge_attr_dict=edge_attr_dict)
            # x_dict = {key: x.relu() for key, x in x_dict.items()}
        return x_dict



# Training and Testing functions

In [67]:
#region Training and Testing functions
def loss_function(output, batch, size, is_train=True):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_ue, num_ap, batch_size = size

    output = torch.reshape(output, (batch_size, num_ue, -1))
    ##
    channel_matrix = batch['ue', 'ap']['edge_attr']
    ##
    power_max = output[:, :,0]
    power = output[:, :, 1] * power_max
    ap_selection = output[:, :, 2] * num_ap
    # power_max = batch['ue']['x'][:, 0]
    # power = batch['ue']['x'][:, 1]
    # ap_selection = batch['ue']['x'][:, 2]
    ##
    ap_selection = ap_selection.int()

    G = torch.reshape(channel_matrix, (-1, num_ap, num_ue))
    # P = torch.reshape(power, (-1, num_ap, num_user)) #* p_max
    P = torch.zeros_like(G, requires_grad=True).clone()
    P[torch.arange(batch_size).unsqueeze(1), ap_selection, torch.arange(num_ue)] = power

    ##
    # new_noise = torch.from_numpy(noise_matrix).to(device)
    desired_signal = torch.sum(torch.mul(P, G), dim=1).unsqueeze(-1)
    G_UE = torch.sum(G, dim=2).unsqueeze(-1)
    all_signal = torch.matmul(P.permute((0,2,1)), G_UE)
    interference = all_signal - desired_signal  # + new_noise
    rate = torch.log(1 + torch.div(desired_signal, interference))
    sum_rate = torch.mean(torch.sum(rate, 1))
    mean_power = torch.mean(torch.sum(P.permute((0,2,1)), 1))

    if is_train:
        return torch.neg(sum_rate )#/ mean_power)
    else:
        return sum_rate #/ mean_power


def train(data_loader):
    model.train()
    device_type = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    total_examples = total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        batch = batch.to(device_type)
        #
        num_ues = batch['ue'].num_nodes
        num_aps = batch['ap'].num_nodes
        num_edges = batch['ue', 'ap'].num_edges
        batch_size = int(num_ues * num_aps / num_edges)
        num_ues = int(num_ues / batch_size)
        num_aps = int(num_aps / batch_size)
        #
        out = model(batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict)
        out = out['ue']
        tmp_loss = loss_function(out, batch, (num_ues, num_aps, batch_size), True)
        tmp_loss.backward()
        optimizer.step()
        total_examples += batch_size
        total_loss += float(tmp_loss) * batch_size

    return total_loss / total_examples


def test(data_loader):
    model.eval()
    device_type = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    total_examples = total_loss = 0
    for batch in data_loader:
        batch = batch.to(device_type)
        #
        num_ues = batch['ue'].num_nodes
        num_aps = batch['ap'].num_nodes
        num_edges = batch['ue', 'ap'].num_edges
        batch_size = int(num_ues * num_aps / num_edges)
        num_ues = int(num_ues / batch_size)
        num_aps = int(num_aps / batch_size)
        #
        out = model(batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict)
        out = out['ue']
        tmp_loss = loss_function(out, batch, (num_ues, num_aps, batch_size), False)
        total_examples += batch_size
        total_loss += float(tmp_loss) * batch_size

    return total_loss / total_examples
#endregion

# Main

In [35]:
K = 3  # number of APs
N = 5  # number of nodes
R = 10  # radius

num_users_features = 3
num_aps_features = 3

num_train = 500  # number of training samples
num_test = 40  # number of test samples

reg = 1e-2
pmax = 1
var_db = 10
var = 1 / 10 ** (var_db / 10)
var_noise = 10e-11

power_threshold = 2.0

X_train, noise_train, pos_train, adj_train, index_train = generate_channels_wsn(K, N, num_train, var_noise, R)
X_test, noise_test, pos_test, adj_test, index_test = generate_channels_wsn(K + 1, N + 1, num_test, var_noise, R)

In [36]:
# Maybe need normalization here
train_data = convert_to_hetero_data(X_train, power_threshold)
test_data = convert_to_hetero_data(X_test, power_threshold)

batchSize = 64

train_loader = DataLoader(train_data, batchSize, shuffle=True, num_workers=1)
test_loader = DataLoader(test_data, batchSize, shuffle=True, num_workers=1)

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = train_data[0]
data = data.to(device)

# model = HGTGNN(data, hidden_channels=64, out_channels=4, num_heads=2, num_layers=1)
# model = model.to(device)

model = RGCN(data, num_layers=2)  # input data for the metadata (list of node types and edge types)
model = model.to(device)

# # # print(data.edge_index_dict)
with torch.no_grad():
    output = model(data.x_dict, data.edge_index_dict, data.edge_attr_dict)
# print(output)

## Training and testing

In [42]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.9)
for epoch in range(1, 100):
    loss = train(train_loader)
    test_acc = test(test_loader)
    if (epoch % 10 == 1):
      print(f'Epoch: {epoch:03d}, Train Loss: {loss:.4f}, Test Reward: {test_acc:.4f}')

Epoch: 001, Train Loss: -1.9734, Test Reward: 2.3622
Epoch: 011, Train Loss: -1.9734, Test Reward: 2.3623
Epoch: 021, Train Loss: -1.9734, Test Reward: 2.3622
Epoch: 031, Train Loss: -1.9734, Test Reward: 2.3621
Epoch: 041, Train Loss: -1.9734, Test Reward: 2.3622
Epoch: 051, Train Loss: -1.9734, Test Reward: 2.3623
Epoch: 061, Train Loss: -1.9734, Test Reward: 2.3621
Epoch: 071, Train Loss: -1.9734, Test Reward: 2.3621
Epoch: 081, Train Loss: -1.9734, Test Reward: 2.3623
Epoch: 091, Train Loss: -1.9734, Test Reward: 2.3618


## Testing Part

In [ ]:
!pip install torchviz

In [ ]:
from torchviz import make_dot

input = data.x_dict
output = model(data.x_dict, data.edge_index_dict, data.edge_attr_dict)
graph = make_dot(output['ue'], params=dict(model.named_parameters()))
graph.view()

'Digraph.gv.pdf'

In [55]:
device_type = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for batch in train_loader:
    optimizer.zero_grad()
    batch = batch.to(device_type)

In [66]:
batch.batch_size

AttributeError: ignored

In [ ]:
optimizer.zero_grad()
batch = batch.to(device_type)
#
num_ues = batch['ue'].x.shape[0]
num_aps = batch['ap'].x.shape[0]
num_edges = batch['ue', 'ap'].edge_index.shape[1]
batch_size = int(num_ues * num_aps / num_edges)
num_ues = int(num_ues / batch_size)
num_aps = int(num_aps / batch_size)
#
out = model(batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict)
out = out['ue']
tmp_loss = loss_function(out, batch, (num_ues, num_aps, batch_size), True)
tmp_loss.backward()
print(batch['ue'].x.requires_grad)
optimizer.step()
total_examples += batch_size
total_loss += float(tmp_loss) * batch_size

False


In [43]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.9)
for epoch in range(1, 2):
  model.train()
  device_type = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  total_examples = total_loss = 0
  for batch in train_loader:
      optimizer.zero_grad()
      batch = batch.to(device_type)
      break

In [45]:
#
num_ues = batch['ue'].x.shape[0]
num_aps = batch['ap'].x.shape[0]
num_edges = batch['ue', 'ap'].edge_index.shape[1]
batch_size = int(num_ues * num_aps / num_edges)
num_ues = int(num_ues / batch_size)
num_aps = int(num_aps / batch_size)
#
out = model(batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict)
batch = batch
output = out['ue']
size = (num_ues, num_aps, batch_size)

In [52]:
output.shape

torch.Size([320, 3])

In [54]:
num_ue, num_ap, batch_size = size
output = torch.reshape(output, (batch_size, num_ue, -1))
channel_matrix = batch['ue', 'ap']['edge_attr']
power_max = output[:, :,0]
power = output[:, :, 1] * power_max
ap_selection = output[:, :, 2] * num_ap
ap_selection = ap_selection.int()
G = torch.reshape(channel_matrix, (-1, num_ap, num_ue))
P = torch.zeros_like(G, requires_grad=True).clone()
P[torch.arange(batch_size).unsqueeze(1), ap_selection, torch.arange(num_ue)] = power

In [49]:
channel_matrix.shape

torch.Size([960, 1])

In [ ]:
#region Training and Testing functions
def loss_function(output, batch, size, is_train=True):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_ue, num_ap, batch_size = size

    output = torch.reshape(output, (batch_size, num_ue, -1))
    ##
    channel_matrix = batch['ue', 'ap']['edge_attr']
    ##
    power_max = output[:, :,0]
    power = output[:, :, 1] * power_max
    ap_selection = output[:, :, 2] * num_ap
    ##
    ap_selection = ap_selection.int()

    G = torch.reshape(channel_matrix, (-1, num_ap, num_ue))
    P = torch.zeros_like(G, requires_grad=True).clone()
    P[torch.arange(batch_size).unsqueeze(1), ap_selection, torch.arange(num_ue)] = power

    ##
    # new_noise = torch.from_numpy(noise_matrix).to(device)
    desired_signal = torch.sum(torch.mul(P, G), dim=1).unsqueeze(-1)
    G_UE = torch.sum(G, dim=2).unsqueeze(-1)
    all_signal = torch.matmul(P.permute((0,2,1)), G_UE)
    interference = all_signal - desired_signal  # + new_noise
    rate = torch.log(1 + torch.div(desired_signal, interference))
    sum_rate = torch.mean(torch.sum(rate, 1))
    mean_power = torch.mean(torch.sum(P.permute((0,2,1)), 1))

    if is_train:
        return torch.neg(sum_rate )#/ mean_power)
    else:
        return sum_rate #/ mean_power

In [ ]:
batch5

HeteroDataBatch(
  ue={
    x=[25, 3],
    batch=[25],
    ptr=[6]
  },
  ap={
    x=[15, 3],
    batch=[15],
    ptr=[6]
  },
  (ue, uplink, ap)={
    edge_attr=[75, 1],
    edge_index=[2, 75]
  },
  (ap, downlink, ue)={
    edge_attr=[75, 1],
    edge_index=[2, 75]
  }
)

In [ ]:
batch2

HeteroDataBatch(
  ue={
    x=[10, 3],
    batch=[10],
    ptr=[3]
  },
  ap={
    x=[6, 3],
    batch=[6],
    ptr=[3]
  },
  (ue, uplink, ap)={
    edge_attr=[30, 1],
    edge_index=[2, 30]
  },
  (ap, downlink, ue)={
    edge_attr=[30, 1],
    edge_index=[2, 30]
  }
)

In [ ]:
batch

HeteroDataBatch(
  ue={
    x=[5, 3],
    batch=[5],
    ptr=[2]
  },
  ap={
    x=[3, 3],
    batch=[3],
    ptr=[2]
  },
  (ue, uplink, ap)={
    edge_attr=[15, 1],
    edge_index=[2, 15]
  },
  (ap, downlink, ue)={
    edge_attr=[15, 1],
    edge_index=[2, 15]
  }
)

In [ ]:
num_ues = batch['ue'].x.shape[0]
num_aps = batch['ap'].x.shape[0]
num_edges = batch['ue', 'ap'].edge_index.shape[1]
batch_size = int(num_ues * num_aps / num_edges)

In [ ]:
batch['ue'].x.shape[0]

5

In [ ]:
batch5['ue', 'ap']['edge_attr']

In [ ]:
ap_selection = torch.tensor([[0, 2, 3, 1, 1],
        [0, 0, 1, 3, 2]])
power = torch.tensor([[1.2, 0.9, 1.0, 0.5, 1.3],
        [1.0, 1.9, 1.5, 1.1, 1.2]])

In [ ]:
P = torch.zeros(2, 4, 5)
index = torch.arange(5)

In [ ]:
ap_selection.shape

torch.Size([2, 5])

In [ ]:
P[:, ap_selection[index], index] = power

IndexError: ignored

In [ ]:
for j in range(2):
  for i in range(5):
    P[j, ap_selection[j,i], i] = power[j, i]

In [ ]:
P[torch.arange(2).unsqueeze(1), ap_selection, torch.arange(5)] = power

In [ ]:
P

tensor([[[1.2000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.5000, 1.3000],
         [0.0000, 0.9000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.0000, 0.0000, 0.0000]],

        [[1.0000, 1.9000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.5000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 1.2000],
         [0.0000, 0.0000, 0.0000, 1.1000, 0.0000]]])

In [ ]:
P[torch.arange(2).unsqueeze(-1), ap_selection, index] = power.unsqueeze(-1)

RuntimeError: ignored

In [ ]:
import torch

# Assuming power is a tensor of shape torch.Size([2, 5])
power = torch.randn(2, 5)

# Assuming ap_selection is a tensor of indices for the second dimension (shape torch.Size([2, 5]))
ap_selection = torch.tensor([[0, 1, 2, 3, 1],
                            [3, 0, 2, 1, 0]])

# Generate the index tensor with torch.arange(5)
index = torch.arange(5)

# Create P with the desired shape (torch.Size([2, 5, 4]))
P = torch.zeros(2, 4, 5)

# Use advanced indexing and broadcasting to assign power to P
P[torch.arange(2).unsqueeze(1), ap_selection, index] = power

print("P tensor:")
print(P)

P tensor:
tensor([[[ 2.3820,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.1026,  0.0000,  0.0000,  0.0937],
         [ 0.0000,  0.0000, -0.5394,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000, -0.1625,  0.0000]],

        [[ 0.0000, -0.1000,  0.0000,  0.0000,  1.1843],
         [ 0.0000,  0.0000,  0.0000, -0.3202,  0.0000],
         [ 0.0000,  0.0000,  0.9463,  0.0000,  0.0000],
         [ 0.4051,  0.0000,  0.0000,  0.0000,  0.0000]]])
